In [20]:
import os, glob 
from enum import Enum
from scikits.audiolab import Sndfile
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
import pandas as pd
import  numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix
import os


os.chdir(os.path.expanduser('~/Desktop/genres/'))
no_of_audio = len(glob.glob('*/*.au'))
Genre = Enum('Genre','classical jazz pop rock')
data = [None]*no_of_audio
data_with_frames = {}
labels = [None]*no_of_audio
file_index = {}
index = 0;


hop_size = 0.01
frame_size = 0.025

for dir in glob.glob("*"):
    if(os.path.isdir(dir) and dir in Genre.__members__):
        for file in glob.glob(dir+"/*.au"):
            f = Sndfile(file,'r')
            Fs = f.samplerate
            audio = f.read_frames(f.nframes)
            label = Genre[dir].value
            features = audioFeatureExtraction.stFeatureExtraction(audio,Fs,Fs*frame_size,Fs*frame_size)
            no_of_features = features.shape[1]
            mean_of_features =np.mean(features,axis=1)
            features_with_label = np.concatenate((mean_of_features,[label]),axis=0)
            new_row = pd.DataFrame([features_with_label],columns=['ZCR']+['EnergyEntropy']+['SpectralEntropy']+['Flux']+['MFCC']*13+['Label'])
            data_with_frames[file] = new_row
            file_index[file] = index
            data[index] = mean_of_features; labels[index] = label
            index=index+1







In [47]:

train_data  = [];
train_data_label = [];
data_panel = pd.Panel.from_dict(data_with_frames)
for i in range(4):
    train_data = train_data+data[i*100:i*100+50]
    train_data_label = train_data_label+labels[i*100:i*100+50]

clf = svm.SVC(C=4.5,kernel='linear',decision_function_shape='ovo')
clf.fit(train_data,train_data_label)
test_data = []
test_data_label = []
for i in range(4):
    test_data = test_data+data[i*100+50:i*100+75]
    test_data_label = test_data_label+labels[i*100+50:i*100+75]
    
predict_label=clf.predict(test_data)
result = confusion_matrix(test_data_label,predict_label)

In [11]:
predict_label




array([4, 4, 4, 4, 3, 2, 3, 3, 4, 2, 4, 4, 2, 4, 3, 2, 2, 2, 4, 4, 4, 4, 4,
       4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 4, 3,
       2, 3, 3, 3, 2, 3, 2, 2, 4, 1, 2, 2, 2, 4, 4, 2, 2, 4, 2, 1, 2, 2, 1,
       4, 3, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 2, 1, 1, 1])

In [12]:
test_data_label

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [170]:
clf.fit(data[75:125],labels[75:125])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [172]:
m = clf.predict(data[1:75]+data[125:200])

In [175]:
k = labels[1:75]+labels[125:200]

In [176]:
m ==k

array([ True,  True, False, False,  True,  True, False,  True, False,
        True,  True, False, False,  True,  True,  True, False, False,
        True, False, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False, False, False,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True, False,
        True,  True, False,  True, False,  True, False,  True,  True,
        True, False, False,  True,  True,  True,  True,  True, False,
        True,  True,  True, False, False,  True, False, False, False,
        True, False, False,  True, False,  True,  True,  True,  True,
       False, False, False,  True,  True, False, False,  True, False,
        True,  True, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True,  True,  True, False, False,  True, False,
       False,  True,

In [178]:
sum(m!=k)

57

In [153]:
for index,file in enumerate(glob.glob('*')):
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [23]:
a= np.array([[[1,2],[3,4]],[[3,5],[4,7]]])


In [29]:
b = a.flatten()

In [43]:
m = [[1,2]];
m=m+[2,4]

In [44]:
m

[[1, 2], 2, 4]

In [40]:
m

[1, 2, 2, 4]

In [129]:
l = [0,1]

In [131]:
clf=svm.SVC()

In [133]:
m


[[0, 0, 0], [1, 1, 1]]

In [103]:
data.values

<function values>

In [105]:
data.values().shaPE

AttributeError: 'list' object has no attribute 'shaPE'

In [108]:
len(data.values()[1])

15

In [109]:
clf = svm.SVC()

ValueError: setting an array element with a sequence.

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1